    This routine is 3 of 4. To be run on flat frames only. It will fit a low-order function to the overscan region, subtract the function from the serial direction of the data, then trim off the overscan region. It subtracts a master bias frame, then subtracts an exposure-scale master dark image. Lastly it combines the flat field images, sorted by filter, into master flat images.

In [1]:
import sys,os,scipy,glob,shutil

In [2]:
from pylab import *

In [3]:
from astropy import units as un

In [4]:
import ccdproc

In [5]:
from ccdproc import ImageFileCollection, Combiner

In [6]:
keys=['imagetyp','filter','exptime'] #keep track of just these keywords

In [7]:
os.chdir('../APO_Data/APO-10-27-17/Q4JH07/UT171028/')

In [8]:
print(os.getcwd())

/home/strolger/APO_Data/APO-10-27-17/Q4JH07/UT171028


In [9]:
flats=glob.glob('flat_?.????.fits')

In [10]:
from astropy.modeling import models

In [11]:
poly_model = models.Polynomial1D(3) #a low order fit

In [12]:
flats_dic={}

In [13]:
master_bias = ccdproc.CCDData.read('Bias.fits')

In [14]:
master_dark = ccdproc.CCDData.read('Dark.fits')

In [15]:
master_dark.header['exptime']=300.0

In [16]:
print(master_dark.header['exptime'])

300.0


In [17]:
wavebands = []

In [18]:
for flatimg in flats:
    tmpimg = ccdproc.CCDData.read(flatimg,unit='adu')
    wavebands.append(tmpimg.header['filter'])

 [astropy.io.fits.verify]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]


In [19]:
wavebands=list(set(wavebands))

In [20]:
for waveband in wavebands:
    flats_dic[waveband]={}

In [21]:
for flatimg in flats:
    if not os.path.isdir('RAW'):
        os.mkdir('RAW')
        print ('RAW directory created')
    if not os.path.isfile('RAW/%s'%flatimg):
        shutil.copyfile(flatimg,'RAW/%s'%flatimg)
        print('%s copied to RAW' %flatimg)
    
    tmpimg = ccdproc.CCDData.read(flatimg,unit='adu')
    tmplog = {'overscan':'overscan region[:,1029:1050] subtracted','calstat':'O'}
    ns_tmpimg = ccdproc.subtract_overscan(tmpimg,overscan=tmpimg[:,1029:1050],model=poly_model,add_keyword=tmplog)
    tmplog = {'trim':'overscan region trimmed','calstat':'OT'}
    tr_tmpimg = ccdproc.trim_image(ns_tmpimg[:,:1024],add_keyword=tmplog)
    tmplog = {'bias':'Zero image subtracted','calstat':'OTZ'}
    ze_tmpimg = ccdproc.subtract_bias(tr_tmpimg,master_bias,add_keyword=tmplog)
    #tmplog = {'dark':'Dark image subtracted','calstat':'OTZD'}
    #dk_tmpimg = ccdproc.subtract_dark(ze_tmpimg,master_dark,exposure_time='exptime',exposure_unit=un.second, scale=False, add_keyword=tmplog)
    dk_tmpimg = ze_tmpimg        
    #dk_tmpimg.write(flatimg,clobber=True)
    if dk_tmpimg.header['filter'] in wavebands:
            flats_dic[dk_tmpimg.header['filter']][flatimg]=dk_tmpimg

flat_g.0231.fits copied to RAW
flat_g.0223.fits copied to RAW
flat_r.0236.fits copied to RAW
flat_g.0226.fits copied to RAW
flat_g.0222.fits copied to RAW
flat_r.0235.fits copied to RAW


 [astropy.io.fits.verify]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]


flat_r.0234.fits copied to RAW
flat_r.0237.fits copied to RAW
flat_r.0240.fits copied to RAW
flat_r.0241.fits copied to RAW
flat_r.0239.fits copied to RAW
flat_r.0232.fits copied to RAW
flat_g.0230.fits copied to RAW
flat_r.0233.fits copied to RAW
flat_g.0228.fits copied to RAW
flat_r.0238.fits copied to RAW
flat_g.0229.fits copied to RAW
flat_g.0225.fits copied to RAW
flat_g.0227.fits copied to RAW
flat_g.0224.fits copied to RAW


In [22]:
for key in flats_dic.keys():
    print(flats_dic[key].keys())
    combiner = Combiner(flats_dic[key].values())
    # combiner.scaling=array(exptimes)## need to replace with mode of image region
    combiner.sigma_clipping()
    med_dark = combiner.median_combine() # may use a more agressive combine later
    med_dark.write('Flat%s.fits'%('_'.join(key.split())))
    print('Flat%s.fits written'%('_'.join(key.split())))
    [os.remove(x) for x in flats_dic[key].keys()]

dict_keys(['flat_r.0236.fits', 'flat_r.0237.fits', 'flat_r.0233.fits', 'flat_r.0239.fits', 'flat_r.0238.fits', 'flat_r.0240.fits', 'flat_r.0235.fits', 'flat_r.0241.fits', 'flat_r.0234.fits', 'flat_r.0232.fits'])
FlatSDSS_r.fits written
dict_keys(['flat_g.0228.fits', 'flat_g.0226.fits', 'flat_g.0223.fits', 'flat_g.0230.fits', 'flat_g.0224.fits', 'flat_g.0222.fits', 'flat_g.0229.fits', 'flat_g.0231.fits', 'flat_g.0225.fits', 'flat_g.0227.fits'])
FlatSDSS_g.fits written
